In [4]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster

asthma = pd.read_csv("Asthma & Air Quality Data.csv")
asthma.head()

,Borough,Geography - UHF 42,Latitude,Longitude,Air Quality Indicator,"Mean, mcg per cubic meter",Age Group,Asthma ER visits
0,Bronx,Crotona - Tremont,40.846229,-73.890437,Fine Particulate Matter (PM2.5),8.2,0 to 17 years,"3,161"
1,Bronx,Fordham - Bronx Pk,40.866630,-73.883030,Fine Particulate Matter (PM2.5),7.6,0 to 17 years,"3,120"
2,Bronx,High Bridge - Morrisania,40.844632,-73.922661,Fine Particulate Matter (PM2.5),8.2,0 to 17 years,"3,559"
3,Bronx,Hunts Point - Mott Haven,40.820281,-73.883187,Fine Particulate Matter (PM2.5),8.3,0 to 17 years,"2,486"
4,Bronx,Kingsbridge - Riverdale,40.886612,-73.899685,Fine Particulate Matter (PM2.5),7.5,0 to 17 years,336


In [5]:
center = [40.7128, -74.0060]

map = folium.Map(location = center, zoom_start = 10)

# Display the map
map.save('mymap.html')
map

In [18]:
map.choropleth(geo_data="Asthma.geojson",
             #data=Asthma.geojson, # my dataset
             columns=['Geography - UHF 42', 'Asthma ER Visits'], # zip code is here for matching the geojson zipcode, sales price is the column that changes the color of zipcode areas
             key_on='feature.properties.postalCode', # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='')

map.save('mymap.html')
map